In [13]:
%pylab inline

import pyart
import wradlib as wrl
import pandas as pd
import tempfile
import os

import pytz
import datetime as dt

from copy import deepcopy

# disable warnings
import warnings
warnings.filterwarnings('ignore')


Populating the interactive namespace from numpy and matplotlib


In [2]:
import boto3
from botocore.handlers import disable_signing

C:\Users\iac6311\AppData\Local\Continuum\anaconda3\envs\saveurproj\lib\site-packages\botocore\vendored\requests\packages\urllib3\_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, MutableMapping


# Setting up SR parameters

In [8]:
# Space-born precipitation radar parameters
sr_pars = {"trmm": {
    "zt": 402500.,  # orbital height of TRMM (post boost)   APPROXIMATION!
    "dr": 250.,     # gate spacing of TRMM
}, "gpm": {
    "zt": 407000.,  # orbital height of GPM                 APPROXIMATION!
    "dr": 125.,      # gate spacing of GPM
}}

In [9]:
# Set parameters for this procedure

platf = "gpm"                # SR platform/product: one out of ["gpm", "trmm"]
ee = 0                        # elevation index
YEAR = '2015'
sweepnum = ['02','03','04','05'][ee]
radarname = 'SUB'

bw_sr = 0.71                  # SR beam width
zt = sr_pars[platf]["zt"]     # SR orbit height (meters)
dr_sr = sr_pars[platf]["dr"]  # SR gate length (meters)

In [14]:
def read_gpm(filename):

    pr_data = wrl.io.read_generic_hdf5(filename)

    lon = pr_data['NS/Longitude']['data']
    lat = pr_data['NS/Latitude']['data']

    year = pr_data['NS/ScanTime/Year']['data']
    month = pr_data['NS/ScanTime/Month']['data']
    dayofmonth = pr_data['NS/ScanTime/DayOfMonth']['data']
    dayofyear = pr_data['NS/ScanTime/DayOfYear']['data']
    hour = pr_data['NS/ScanTime/Hour']['data']
    minute = pr_data['NS/ScanTime/Minute']['data']
    second = pr_data['NS/ScanTime/Second']['data']
    secondofday = pr_data['NS/ScanTime/SecondOfDay']['data']
    millisecond = pr_data['NS/ScanTime/MilliSecond']['data']
    date_array = zip(year, month, dayofmonth,
                     hour, minute, second,
                     millisecond.astype(np.int32) * 1000)
    pr_time = np.array(
        [dt.datetime(d[0], d[1], d[2], d[3], d[4], d[5], d[6]) for d in
         date_array])

    sfc = pr_data['NS/PRE/landSurfaceType']['data']
    pflag = pr_data['NS/PRE/flagPrecip']['data']

    bbflag = pr_data['NS/CSF/flagBB']['data']
    zbb = pr_data['NS/CSF/heightBB']['data']
    bbwidth = pr_data['NS/CSF/widthBB']['data']
    qbb = pr_data['NS/CSF/qualityBB']['data']
    qtype = pr_data['NS/CSF/qualityTypePrecip']['data']
    ptype = pr_data['NS/CSF/typePrecip']['data']

    quality = pr_data['NS/scanStatus/dataQuality']['data']
    refl = pr_data['NS/SLV/zFactorCorrected']['data']

    # Check for bad data
    if max(quality) != 0:
        raise ValueError('GPM contains Bad Data')

    pflag = pflag.astype(np.int8)

    # Determine the dimensions
    ndim = refl.ndim
    if ndim != 3:
        raise ValueError('GPM Dimensions do not match! Needed 3, given {0}'.format(ndim))

    tmp = refl.shape
    nscan = tmp[0]
    nray = tmp[1]
    nbin = tmp[2]

    # Reverse direction along the beam
    # TODO: Why is this reversed?
    refl = refl[::-1]

    # Change pflag=1 to pflag=2 to be consistent with 'Rain certain' in TRMM
    pflag[pflag == 1] = 2

    # Simplify the precipitation types
    ptype = (ptype/1e7).astype(np.int16)

    # Simplify the surface types
    imiss = (sfc == -9999)
    sfc = (sfc/1e2).astype(np.int16) + 1
    sfc[imiss] = 0
	
	# Missing Data
    refl[refl == -9999.90039062] = np.nan

    # Set a quality indicator for the BB and precip type data
    # TODO: Why is the `quality` variable overwritten?

    quality = np.zeros((nscan, nray), dtype=np.uint8)

    i1 = ((qbb == 0) | (qbb == 1)) & (qtype == 1)
    quality[i1] = 1

    i2 = ((qbb > 1) | (qtype > 2))
    quality[i2] = 2

    gpm_data = {}
    gpm_data.update({'nscan': nscan, 'nray': nray, 'nbin': nbin,
                     'date': pr_time, 'lon': lon, 'lat': lat,
                     'pflag': pflag, 'ptype': ptype, 'zbb': zbb,
                     'bbwidth': bbwidth, 'sfc': sfc, 'quality': quality,
                     'refl': refl})

    return gpm_data


# Define and read data

##  Define GPM Data set

In [10]:
gpm_file = 'C:/Users/iac6311/Documents/Work/Data/GPM/2A-IL-91W43N86W38N.GPM.Ku.V8-20180723.20190526-S170152-E170250.029776.V06A.HDF5'

In [15]:
sr_data = wrl.io.read_gpm(gpm_file)


# Download radar data
NEXRAD data is downloaded from Amazon S3.

Define function to download data.

In [3]:
def get_radar_scan(station='KLOT', date=None, key_index=-20):
    
    '''
    Function will pull the latest radar scan from any radar site using 
    Amazon S3.
    ----------
    Station = Four letter NEXRAD identifier
              Example: 'KEPZ'
    Date = default is none for current date, else enter date in format "YYYY/MM/DD"
    Ex: date ='2013/11/17
    Key_index = Number of keys you want pulled from most recent scan.
    Ex: key_index = -15 would pull ht most recent 15 scans
    '''
    
    # Creating a bucket and a client to be able to pull data from AWS and setting it as unsigned
    bucket = 'noaa-nexrad-level2'
    s3 = boto3.resource('s3')
    s3.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)
    
    # Connects the bucket create above with radar data
    aws_radar = s3.Bucket(bucket)
    
    # Setting the date and time to current...
    # This will allow for allow the current date's radar scands to be pulled
    if date == None:
        target_string = datetime.datetime.utcnow().strftime('%Y/%m/%d/'+station)
    else:
        target_string = date+'/'+station
    
    for obj in aws_radar.objects.filter(Prefix= target_string):
        '{0}:{1}'.format(aws_radar.name, obj.key)
    my_list_of_keys = [this_object.key for this_object in aws_radar.objects.filter(Prefix= target_string)]
    keys = my_list_of_keys[key_index:]
    newkeys = []
    for key in keys:
        if 'MDM' in key:
            pass
        elif key.endswith('.tar'):
            pass
        else:
            newkeys.append(key)
    #print(newkeys)
    return aws_radar, newkeys

In [17]:
# Setting radar, date of radar scans needed, and key index (amount of items in list)
aws_radar, keys = get_radar_scan(station='KLOT', date='2019/05/26', key_index=-400)

Using the function defined above, we can download the file, save it to a temporary file, read the radar object using pyart, then delete the file to save space.

In [30]:
# loop through the keys by iterating nframe
nframe = 175

# open a temporary local file
localfile = tempfile.NamedTemporaryFile(delete=False)
localfile_name = localfile.name
localfile.close()

# download to temporary file and read to radar object using pyart
aws_radar.download_file(keys[nframe], localfile_name)
radar = pyart.io.read(localfile_name)

# delete temporary file to save space
os.remove(localfile_name)

# Extract reflectivity
The radar object contains reflectivty data, among other variables. We want to extract the reflectivity to compare to GPM reflectivity.

In [31]:
gatefilter = pyart.filters.GateFilter(radar)
# Develop your gatefilter first
# exclude masked gates from the gridding
#gatefilter = pyart.filters.GateFilter(radar)
gatefilter.exclude_transition()
gatefilter.exclude_masked('reflectivity')
# Mask reflectivity
radar.fields["corrected_reflectivity"] = deepcopy(radar.fields["reflectivity"])
radar.fields["corrected_reflectivity"]["data"] = np.ma.masked_where(
    gatefilter._gate_excluded, radar.fields["corrected_reflectivity"]["data"])

### read GR cfradial

In [32]:
date = radar['date']

TypeError: 'Radar' object is not subscriptable

In [36]:
dir(radar)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_sweep_in_range',
 '_dic_info',
 'add_field',
 'add_field_like',
 'altitude',
 'altitude_agl',
 'antenna_transition',
 'azimuth',
 'check_field_exists',
 'drift',
 'elevation',
 'extract_sweeps',
 'fields',
 'fixed_angle',
 'gate_altitude',
 'gate_latitude',
 'gate_longitude',
 'gate_x',
 'gate_y',
 'gate_z',
 'georefs_applied',
 'get_azimuth',
 'get_elevation',
 'get_end',
 'get_field',
 'get_gate_x_y_z',
 'get_nyquist_vel',
 'get_slice',
 'get_start',
 'get_start_end',
 'heading',
 'info',
 'init_gate_altitude',
 'init_gate_longitude_latitude',
 'init_gate_x_y_z',
 'init_rays_per

In [58]:
date = dt.datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')
source = 'reflectivity'

lat = radar.latitude['data'][0]
lon = radar.longitude['data'][0]
alt = radar.altitude['data'][0]

ngate = radar.ngates # number of gates in gr beam
nbeam = radar.nrays # number of rays in gr sweep
r0 = radar.range['data'][0]# range of first gate
dr = radar.range['meters_between_gates']# gate length (m)
a0 = radar.azimuth['data'][0] # azimuth angle of first beam

In [57]:
radar.range

{'units': 'meters',
 'standard_name': 'projection_range_coordinate',
 'long_name': 'range_to_measurement_volume',
 'axis': 'radial_range_coordinate',
 'spacing_is_constant': 'true',
 'comment': 'Coordinate variable for range. Range to center of each bin.',
 'data': array([  2125.,   2375.,   2625., ..., 459375., 459625., 459875.],
       dtype=float32),
 'meters_to_center_of_first_gate': 2125.0,
 'meters_between_gates': 250.0}

In [ ]:
# # number of rays in gr sweep
# try:
#     nray_gr = gr_data['nbeam'].astype("i4")[ee]
# except IndexError:
#     raise RadarFileError
# # number of gates in gr beam
# ngate_gr = gr_data['ngate'].astype("i4")[ee]
# # number of sweeps
# nelev = gr_data['ntilt']
# # elevation of sweep (degree)
# elev_gr = gr_data['elang'][ee]
# # gate length (meters)
# dr_gr = gr_data['dr'][ee]
# # reflectivity array of sweep
# ref_gr = gr_data['refl'][ee]
# # sweep datetime stamp
# date_gr = gr_data['sdate'][ee]
# # range of first gate
# r0_gr = gr_data['r0'][ee]                    
# # azimuth angle of first beam
# a0_gr = gr_data['a0'][ee]
# # Longitude of GR
# lon0_gr = gr_data['lon']
# # Latitude of GR
# lat0_gr = gr_data['lat']
# # Altitude of GR (meters)
# alt0_gr = gr_data['alt']
# # Beam width of GR (degree)
# bw_gr = 1.